In [ ]:
# ensure that any code changes are immediately reflected
%reload_ext autoreload
%autoreload 2

In [ ]:
from mostlyai import MostlyAI
import pandas as pd
import os
os.environ['MOSTLY_AI_PASSWORD'] = 'Mostly01!'
mostly = MostlyAI()

In [ ]:
json_config = """
{
   "name": "titanic",
   "description": null,
   "tables": [
      {
         "name": "titanic",
         "sourceConnectorId": "372e4a44-966b-4420-bd10-f1e847caf22c",
         "location": null,
         "data": null,
         "modelConfiguration": {
            "sampleFraction": 1,
            "maxSampleSize": null,
            "batchSize": null,
            "modelSize": "M",
            "maxTrainingTime": 60,
            "maxEpochs": 100,
            "maxSequenceWindow": 100,
            "enableFlexibleGeneration": true,
            "valueProtection": true,
            "rareCategoryReplacementMethod": "CONSTANT"
         },
         "textModelConfiguration": null,
         "primaryKey": null,
         "foreignKeys": null,
         "columns": [
            {
               "name": "survived",
               "included": true,
               "modelEncodingType": "NUMERIC_AUTO"
            },
            {
               "name": "pclass",
               "included": true,
               "modelEncodingType": "NUMERIC_AUTO"
            },
            {
               "name": "sex",
               "included": true,
               "modelEncodingType": "CATEGORICAL"
            },
            {
               "name": "age",
               "included": true,
               "modelEncodingType": "NUMERIC_AUTO"
            },
            {
               "name": "sibsp",
               "included": true,
               "modelEncodingType": "NUMERIC_AUTO"
            },
            {
               "name": "parch",
               "included": true,
               "modelEncodingType": "NUMERIC_AUTO"
            },
            {
               "name": "fare",
               "included": true,
               "modelEncodingType": "NUMERIC_AUTO"
            },
            {
               "name": "embarked",
               "included": true,
               "modelEncodingType": "CATEGORICAL"
            }
         ]
      }
   ]
}
"""

In [ ]:
import json
config = json.loads(json_config)
config

In [ ]:
config["tables"][0]["name"] = 'titanic_dataset'
config["tables"][0]["modelConfiguration"]["maxSampleSize"] = 20
config["tables"][0]["modelConfiguration"] |= {"batchSize": 32, "maxEpochs": 5, "sampleFraction": None}
config["tables"][0]["data"] = "/Users/mplatzer/github/public-demo-data/titanic/titanic.csv"
config

In [ ]:
mostly.train(config)

In [ ]:
g = mostly.generators.get('82500232-6e90-4c33-bc77-5e5e61f36581')
g.to_dict()

In [ ]:
sd = mostly.generate(g)

In [ ]:
mostly.connect(config)

mostly.connectors.get(id)
mostly.connectors.list()
mostly.connectors.create(config)
c.delete()
c.update()
c.config()
c.locations()

In [ ]:
mostly.train(data_or_config, start=True, wait=True)

mostly.generators.get(id)
mostly.generators.list()
mostly.generators.create(config)
g.training.start()
g.training.progress()
g.training.cancel()
#g.training.stop()
g.training.wait()
g.delete()
g.update(config)
g.config()

In [ ]:
mostly.generate(generator, config, seed=seed, start=True, wait=True)

mostly.synthetic_datasets.get(id)
mostly.synthetic_datasets.list()
mostly.synthetic_datasets.create(config)
sd.generation.start()
sd.generation.progress()
sd.generation.cancel()
sd.generation.wait()
sd.update(config)
sd.config()
sd.data()
sd.download(file, artifact)
sd.delete()


## TITANIC

In [ ]:
data_fn = 'https://raw.githubusercontent.com/mostly-ai/public-demo-data/dev/titanic/titanic.csv'
g = mostly.train(data_fn)

In [ ]:
sd = mostly.generate(g)

In [ ]:
dfs = sd.data()
dfs

## CENSUS

In [ ]:
census_df = pd.read_csv('https://github.com/mostly-ai/public-demo-data/raw/dev/census/census.csv.gz').sample(n=1_000)
config = {
    'name': 'US census',
    'tables': [{"name": "census", "data": census_df}],
}
g = mostly.train(config)

In [ ]:
#seed = pd.DataFrame({'sex': ['Male', 'Female'] * 100, 'age': [49] * 200})
#sd = client.synthetic_datasets.create(generatorId=g.id, tables=[{'name': 'users', 'sampleSeedData': seed}])

## CDNOW

In [ ]:
g = mostly.train(
    {
        "name": "CDNOW",
        "tables": [
            {
                "name": "users", 
                "data": "https://github.com/mostly-ai/public-demo-data/raw/dev/cdnow/users.csv.gz", 
                "primaryKey": "id"
            },
            {
                "name": "purchases",
                "data": "https://github.com/mostly-ai/public-demo-data/raw/dev/cdnow/purchases.csv.gz", 
                "foreignKeys": [{"column": "users_id", "referencedTable": "users", "isContext": True}]
            }
        ]
    }
)

In [ ]:
g = mostly.generators.get('8ff1e247-2582-4ddf-9903-58f9fa5cfc1f')
acc = g.tables[1].model.metrics.accuracy.univariate
print(f"accuracy: {acc:.2%}")

In [ ]:
sd = mostly.generate(g)

In [ ]:
sd = mostly.synthetic_datasets.get('625fcb54-de26-4dd6-a406-eac249cdcf04')

In [ ]:
dfs = sd.data()
dfs

# clone a generator

In [ ]:
g = mostly.generators.get('6422bf4b-1e3f-4b8e-bc33-f1a33dac8925')
print(f"id:{str(g.id)}, name:{g.name}, status:{g.training_status.value}")

In [ ]:
# clone a generator, and re-provide the data
config = g.to_dict()
config['tables'][0]['data'] = 'https://github.com/mostly-ai/public-demo-data/raw/dev/census/census.csv.gz'
g2 = mostly.train(config, start=False)

In [ ]:
print(f"id:{str(g2.id)}, name:{g2.name}, status:{g2.training_status.value}")

In [ ]:
config = mostly.generators.get(g2.id).to_dict()
config['tables'][0]['modelConfiguration'] |= {'maxTrainingTime': 2, 'maxEpochs': 5}
g2 = g2.update(**config)

In [ ]:
g2.training.start()
g2 = g2.training.wait(interval=5)
print(f"{g2.training_status.value=}")

In [ ]:
g2 = mostly.generators.get('6422bf4b-1e3f-4b8e-bc33-f1a33dac8925')
train_step = [step for step in g2.training.progress().steps if step.step_code.value=='TRAIN_MODEL'][0]
for ckpt in train_step.messages[:5]:
    print(f"epoch {ckpt['epoch']}: {ckpt['val_loss']:0.4f}")

## list generators

In [ ]:
for g in mostly.generators.list():
    print(f"name:{g.name}, status:{g.training_status.value}, accuracy:{g.accuracy}")